# Mixtral in Colab

Welcome! In this notebook you can run [Mixtral8x7B-Instruct](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) with decent generation speed **right in Google Colab or on a consumer-grade GPU**. This was made possible by quantizing the original model in mixed precision and implementing a MoE-specific offloading strategy.

To learn more, read our [tech report](https://arxiv.org/abs/2312.17238) or check out the [repo](https://github.com/dvmazur/mixtral-offloading) on GitHub.

One will need approximately 16 GB of VRAM and 11 GB of RAM to run this notebook and generate somewhat long texts.


<details>

<summary>How to balance between RAM and GPU VRAM usage</summary>

You can balance between RAM and GPU VRAM usage by changing <code>offload_per_layer</code> variable in the <a href="#scrollTo=_mIpePTMFyRY&line=10&uniqifier=1">Initialize model</a> section. Increasing <code>offload_per_layer</code> will decrease GPU VRAM usage, increase RAM usage and decrease generation speed. Decreasing <code>offload_per_layer</code> will have the opposite effect.

Note that this notebook should run normally in Google Colab with <code>offload_per_layer = 4</code>, but may crush with other values. However, if you run this somewhere else, you're free to play with this variable.
</details>

## Install and import libraries

In [2]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

# !git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
# !cd ../mixtral-offloading && pip install -q -r requirements.txt
# !huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo
# !huggingface-cli download mistralai/Mixtral-8x7B-Instruct-v0.1 --quiet --local-dir Mixtral-8x7B-Instruct-v0.1

clear_output()

In [3]:
import sys

sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/home/cc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cc/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/cc/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/cc/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [1]:
%load_ext autoreload
%autoreload 2

## Initialize model

In [4]:
quantized = False

if quantized == False:
    state_path = "Mixtral-8x7B-Instruct-v0.1"
    model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
else:
    state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"
    model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
# offload_per_layer = 4
offload_per_layer = 7
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)

if quantized:
    quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)
else:
    quant_config = None


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

/home/cc/.local/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


352321536
ExpertCache: loading main module 0 on GPU...
352321536
ExpertCache: loading main module 1 on GPU...
352321536
ExpertCache: loading main module 2 on GPU...
352321536
ExpertCache: loading main module 3 on GPU...
352321536
ExpertCache: loading main module 4 on GPU...
352321536
ExpertCache: loading main module 5 on GPU...
352321536
ExpertCache: loading main module 6 on GPU...
352321536
ExpertCache: loading main module 7 on GPU...
352321536
ExpertCache: loading main module 8 on GPU...
352321536
ExpertCache: loading main module 9 on GPU...
352321536
ExpertCache: loading main module 10 on GPU...
352321536
ExpertCache: loading main module 11 on GPU...
352321536
ExpertCache: loading main module 12 on GPU...
352321536
ExpertCache: loading main module 13 on GPU...
352321536
ExpertCache: loading main module 14 on GPU...
352321536
ExpertCache: loading main module 15 on GPU...
352321536
ExpertCache: loading main module 16 on GPU...
352321536
ExpertCache: loading main module 17 on GPU...
35

Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

352321536
352321536
352321536
352321536
352321536
352321536
352321536


Loading experts:   3%|▎         | 1/32 [00:18<09:29, 18.37s/it]

...

352321536
352321536
352321536
352321536
352321536
352321536
352321536
352321536


Loading experts:  97%|█████████▋| 31/32 [11:50<00:25, 25.63s/it]

352321536
352321536
352321536
352321536
352321536
352321536
352321536
352321536


Loading experts: 100%|██████████| 32/32 [12:25<00:00, 23.30s/it]

352321536


## Run the model

In [5]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0
while True:
  print("User: ", end="")
  user_input = input()
  print("\n")

  user_entry = dict(role="user", content=user_input)
  input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

  if past_key_values is None:
    attention_mask = torch.ones_like(input_ids)
  else:
    seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
    attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

  print("Mixtral: ", end="")
  result = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    past_key_values=past_key_values,
    streamer=streamer,
    do_sample=True,
    temperature=0.9,
    top_p=0.9,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    return_dict_in_generate=True,
    output_hidden_states=True,
  )
  print("\n")

  sequence = result["sequences"]
  past_key_values = result["past_key_values"]

User: 

Mixtral: unionื obtainließResultцоimes励 PetDrawable老Metadataenguimesrmcolaatos엔 cinULE德ermanLongDrawablencia 

KeyboardInterrupt: 